# 反爬：登入授權模擬

* 了解「登入權限機制」的反爬蟲機制
* 「登入權限機制」反爬蟲的因應策略

## 作業目標

* 找一個需要登入的網站試試看，並說明思考流程
（如果不知道要用哪個網站的話，可以試試看 https://github.com/new 網址，未登入時會被導向登入頁）



In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
login_url = "https://github.com/login"

In [3]:
from lxml import html

In [4]:
session_requests = requests.session()
result = session_requests.get(login_url)
tree = html.fromstring(result.text)
authenticity_token = list(set(tree.xpath("//input[@name='authenticity_token']/@value")))[0]
authenticity_token

'NCu9L31+86/ReXVf3roYzGnwdN8Bs0Hj3CgLq3FfBbOo1FBBCCiYIVXQ1W37jjeZRa+i/8zOWA0UJy/mRl5O8Q=='

In [5]:
for cookie in result.headers['Set-Cookie'].split(";"):
    if "_gh_sess" in cookie:
        for sess in cookie.strip().split(","):
            if "_gh_sess" in sess:
                gh_sess = sess.strip()
gh_sess

'_gh_sess=clE3dW9haU5NNWJVU0M1dURHUDh0Z2NaWVloYno3bDVGVENRNHVoNUVaZEpvRUtqK0wwU1hVQ3JlTk9JSndkQWZ4dkdqM2FKeEVjR2ZzRXNySGtmaXNTR2tRZUU0Y2FIQU4wR3RiQm5EbzVTUDdVWW5mTHl3dXQ4VzBheWZ4N0RXSmQwU3ZZN3g0RTJEMHhRNUVvOWxXclRyL3FQd3JWYk1UaWh0VjdKMUpDU2hRc214Z2JSMEdMSUtoa0RnL2lmM3Vkd0ZYNVEzZ1BoNlZURlEwaktWT2pVOHNNdHJlYk8xOHI3SDdtbXpVRmF3ZVFLRithMHVXMkk4MGRzZkRUTC0tSERQbmt1eE9Sd0NKZStZdlJtblJSZz09--1955183950580e658aff169fe69987bc93470eee'

In [6]:
timestamp = list(set(tree.xpath("//input[@name='timestamp']/@value")))[0]
timestamp

'1578754238637'

In [7]:
timestamp_secret = list(set(tree.xpath("//input[@name='timestamp_secret']/@value")))[0]
timestamp_secret

'40cf571d6398133c5709a2b17df4dc38a1b5cda1721ad63e4057042610dd846e'

In [8]:
import re

In [9]:
soup_login = BeautifulSoup(result.text,'html.parser')
required_field = soup_login.find("input",attrs={'name':re.compile("required_field")})['name']
required_field

'required_field_53f0'

In [10]:
with open('github.txt', newline='',encoding="utf8") as gitfile:
    rows = gitfile.read().split()
for row in rows:
    if "username" in row:
        username = row.split(":")[-1]
    if "pass" in row:
        password = row.split(":")[-1]

In [11]:
payload={
    'commit':'Sign in'
    ,'utf8':'✓'
    ,'authenticity_token':authenticity_token
    ,'ga_id':'1589911901.1578669634'
    ,'login':username
    ,'password':password
    ,'webauthn-support':'supported'
    ,'webauthn-iuvpaa-support':'unsupported'
    ,required_field:''
    ,'timestamp':timestamp
    ,'timestamp_secret':timestamp_secret
}

In [12]:
print("authenticity_token",authenticity_token)
print("required_field",required_field)
print("timestamp",timestamp)
print("timestamp_secret",timestamp_secret)

authenticity_token NCu9L31+86/ReXVf3roYzGnwdN8Bs0Hj3CgLq3FfBbOo1FBBCCiYIVXQ1W37jjeZRa+i/8zOWA0UJy/mRl5O8Q==
required_field required_field_53f0
timestamp 1578754238637
timestamp_secret 40cf571d6398133c5709a2b17df4dc38a1b5cda1721ad63e4057042610dd846e


In [13]:
header={
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    ,'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7'
    ,'Cache-Control':'max-age=0'
    ,'Connection':'keep-alive'
    ,'Cookie':'_octo=GH1.1.30085354.1578669636; _ga=GA1.2.1589911901.1578669634; tz=Asia%2FTaipei; _device_id=0ad2a32ca9452275417b65fcb0387a21; logged_in=no;' + gh_sess
    ,'Content-Length':'420'
    ,'Content-Type':'application/x-www-form-urlencoded'
    ,'Host':'github.com'
    ,'Origin':'https://github.com'
    ,'Referer':'https://github.com/login'
    ,'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

In [14]:
header['Cookie']

'_octo=GH1.1.30085354.1578669636; _ga=GA1.2.1589911901.1578669634; tz=Asia%2FTaipei; _device_id=0ad2a32ca9452275417b65fcb0387a21; logged_in=no;_gh_sess=clE3dW9haU5NNWJVU0M1dURHUDh0Z2NaWVloYno3bDVGVENRNHVoNUVaZEpvRUtqK0wwU1hVQ3JlTk9JSndkQWZ4dkdqM2FKeEVjR2ZzRXNySGtmaXNTR2tRZUU0Y2FIQU4wR3RiQm5EbzVTUDdVWW5mTHl3dXQ4VzBheWZ4N0RXSmQwU3ZZN3g0RTJEMHhRNUVvOWxXclRyL3FQd3JWYk1UaWh0VjdKMUpDU2hRc214Z2JSMEdMSUtoa0RnL2lmM3Vkd0ZYNVEzZ1BoNlZURlEwaktWT2pVOHNNdHJlYk8xOHI3SDdtbXpVRmF3ZVFLRithMHVXMkk4MGRzZkRUTC0tSERQbmt1eE9Sd0NKZStZdlJtblJSZz09--1955183950580e658aff169fe69987bc93470eee'

In [15]:
rs = requests.session()
res = rs.post("https://github.com/session",verify = False, data = payload, headers = header)

D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [16]:
res.status_code

200

In [17]:
#res = rs.get('https://github.com/new', verify = False)
soup = BeautifulSoup(res.text,'html.parser')
print(soup.text)
















GitHub





























































Skip to content




































        Search
      

        All GitHub
      
↵


      Jump to
      ↵






No suggested jump to results















        Search
      

        All GitHub
      
↵


      Jump to
      ↵















        Search
      

        All GitHub
      
↵


      Jump to
      ↵









 



      Dashboard


    Pull requests


    Issues



        Marketplace



    Explore



      kevinharryshen




        Sign out
      
















 



  New repository


    Import repository
  

  New gist


    New organization
  

    New project
  











Signed in as kevinharryshen













Set status










Edit status


















Could not update your status, please try again.


              80 remaining
            




Suggestions:




🌴


                          On vacation
                        



🤒


           

In [19]:
res.headers['Set-Cookie'].split(";")

['user_session=Wt5ARHFK3DzJAT4bWij8hrFKq1L26GZ7MaA9a2QOHWhqgvl4',
 ' path=/',
 ' expires=Sat, 25 Jan 2020 14:51:00 -0000',
 ' secure',
 ' HttpOnly, __Host-user_session_same_site=Wt5ARHFK3DzJAT4bWij8hrFKq1L26GZ7MaA9a2QOHWhqgvl4',
 ' path=/',
 ' expires=Sat, 25 Jan 2020 14:51:00 -0000',
 ' secure',
 ' HttpOnly',
 ' SameSite=Strict, has_recent_activity=1',
 ' path=/',
 ' expires=Sat, 11 Jan 2020 15:51:00 -0000, _octo=GH1.1.1495006576.1578754261',
 ' domain=.github.com',
 ' path=/',
 ' expires=Tue, 11 Jan 2022 14:51:01 -0000, _gh_sess=YlRJb09FVFNUdVJRQU1Xc0NkZGlVbFNpZGhLdVc3bTNSRm1rRklGclpVZW8ybmtwOWpyU3hlYmkxbk5sQ2ZISXJKR1FSQU82cEIxNUNWaEdxWVh6bEdZMVE2aE5kRjJacjB6dWZCRmxaSjVFVmowaEpkUFpScFJuNmE0VXdpZlMrUWJuM3lObnZJRTdWY0JpN2tWVjVsTEJLOThycmRJdktOYm85TFN2UTRtb2xIcHJtM1BWSVdtY0RnYzRWUFB0T0JEL29kL1Q5bDlMQzI4SGc2RERMZ2ovL3hJdXlTdUkzQkZmd3liSzdIVmtkc1pCUUJWbnd3NFZ1Qkp0WjlmT3M1cUVRUENJV3pZdFZnMXdQY0tHbDRQRDZ4OEpZTERkV2VxaFFMVERzaHdHbkZTZGVGVEhCbGtjSGNCMVJ0NlNsVE5QYlZ2SER4RGpwMm1Kdzd1T21uRUpkdkk

In [18]:
res_new = rs.get(' https://github.com/new',verify = False)
soup_new = BeautifulSoup(res_new.text,'html.parser')
print(soup_new.text) 

D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

















Create a New Repository


























































Skip to content































        Search
      

        All GitHub
      
↵


      Jump to
      ↵






No suggested jump to results















        Search
      

        All GitHub
      
↵


      Jump to
      ↵















        Search
      

        All GitHub
      
↵


      Jump to
      ↵









 



    Pull requests


    Issues



        Marketplace



    Explore












 



  New repository


    Import repository
  

  New gist


    New organization
  

    New project
  











Signed in as kevinharryshen













Set status










Edit status


















Could not update your status, please try again.


              80 remaining
            




Suggestions:




🌴


                          On vacation
                        



🤒


                          Out sick
                        





🏠


       